In [1]:
import geopyspark as gps
import matplotlib.pyplot as plt

from colortools import Color
from pyspark import SparkContext
from functools import partial

import fiona
import pyproj

from shapely.geometry import MultiPoint, MultiLineString, shape
from shapely.ops import transform
from geonotebook.wrappers import VectorData, TMSRasterData
%matplotlib inline

In [2]:
!curl -o /tmp/cropped.tif https://raw.githubusercontent.com/tpolong/geodocker/master/cropped.tif

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5631k  100 5631k    0     0   128k      0  0:00:43  0:00:43 --:--:--  253k


In [3]:
conf = gps.geopyspark_conf(master="local[*]", appName="visualization")
pysc = SparkContext(conf=conf)

In [4]:
raster_layer = gps.geotiff.get(layer_type=gps.LayerType.SPATIAL, uri="/tmp/cropped.tif")
tiled_layer = raster_layer.tile_to_layout(layout=gps.GlobalLayout(), target_crs=3857)

In [5]:
pyramided = tiled_layer.pyramid()
pyramided

Pyramid(layer_type=LayerType.SPATIAL, max_zoom=11, num_levels=12, is_cached=False)

In [6]:
gps.Pyramid([tiled_layer.tile_to_layout(gps.GlobalLayout(zoom=x)) for x in range(0, 13)])

Pyramid(layer_type=LayerType.SPATIAL, max_zoom=12, num_levels=13, is_cached=False)

In [7]:
gps.Pyramid({x: tiled_layer.tile_to_layout(gps.GlobalLayout(zoom=x)) for x in range(0, 13)})

In [8]:
hist = pyramided.get_histogram()
hist

In [9]:
pyramided + 1

Pyramid(layer_type=LayerType.SPATIAL, max_zoom=11, num_levels=12, is_cached=False)

In [10]:
(2 * (pyramided + 2)) / 3

Pyramid(layer_type=LayerType.SPATIAL, max_zoom=11, num_levels=12, is_cached=False)

In [11]:
small_pyramid = gps.Pyramid({x: tiled_layer.tile_to_layout(gps.GlobalLayout(zoom=x)) for x in range(0, 5)})
small_pyramid

Pyramid(layer_type=LayerType.SPATIAL, max_zoom=4, num_levels=5, is_cached=False)

In [12]:
pyramided + small_pyramid

Pyramid(layer_type=LayerType.SPATIAL, max_zoom=4, num_levels=5, is_cached=False)

In [13]:
color_ramp = gps.get_colors_from_matplotlib(ramp_name="viridis")
color_ramp

[1140937983,
 1141003775,
 1141069823,
 1157978367,
 1158044415,
 1158175743,
 1175018751,
 1175150335,
 1175216127,
 1175347711,
 1192190719,
 1192322047,
 1192388095,
 1192519423,
 1192585215,
 1192651263,
 1192782591,
 1209625599,
 1209691391,
 1209822975,
 1209888767,
 1209954559,
 1210085887,
 1210151679,
 1210217471,
 1210283263,
 1193637375,
 1193703167,
 1193768959,
 1193834751,
 1193966079,
 1194031871,
 1194097663,
 1177386239,
 1177517311,
 1177583103,
 1177648895,
 1160937471,
 1161068543,
 1161134335,
 1161200127,
 1144488447,
 1144619775,
 1127908351,
 1127973887,
 1128039679,
 1111327999,
 1111393791,
 1111524863,
 1094813439,
 1094878975,
 1078167551,
 1078233087,
 1061521407,
 1061652735,
 1044941055,
 1045006847,
 1028295167,
 1028360703,
 1028426239,
 1011714815,
 1011780351,
 995134207,
 995199743,
 978488319,
 978553855,
 961842175,
 961907711,
 945196031,
 945261823,
 928550143,
 928615679,
 911903999,
 911969535,
 895257855,
 895323391,
 878611967,
 878677503,
 8

In [14]:
gps.get_colors_from_matplotlib(ramp_name="hot", num_colors=150)

[167772415,
 218104063,
 301990143,
 385876223,
 436207871,
 520093951,
 603980031,
 704643327,
 738197759,
 822083839,
 922747135,
 956301567,
 1056964863,
 1140850943,
 1224737023,
 1275068671,
 1358954751,
 1442840831,
 1493172479,
 1577058559,
 1660944639,
 1761607935,
 1795162367,
 1879048447,
 1979711743,
 2013266175,
 2113929471,
 2197815551,
 2281701631,
 2332033279,
 2415919359,
 2499805439,
 2550137087,
 2634023167,
 2717909247,
 2818572543,
 2852126975,
 2936013055,
 3036676351,
 3120562431,
 3170894079,
 3254780159,
 3338666239,
 3388997887,
 3472883967,
 3556770047,
 3640656127,
 3690987775,
 3774873855,
 3875537151,
 3909091583,
 3992977663,
 4093640959,
 4177527039,
 4227858687,
 4278321407,
 4278649087,
 4278845695,
 4279173375,
 4279501055,
 4279894271,
 4280025343,
 4280353023,
 4280746239,
 4280877311,
 4281270527,
 4281598207,
 4281925887,
 4282122495,
 4282450175,
 4282777855,
 4282974463,
 4283302143,
 4283629823,
 4284023039,
 4284154111,
 4284481791,
 4284875007

In [15]:
colors = [Color('green'), Color('red'), Color('blue')]
colors

[Color(0, 128, 0, 255), Color(255, 0, 0, 255), Color(0, 0, 255, 255)]

In [16]:
colors_color_ramp = gps.get_colors_from_colors(colors=colors)
colors_color_ramp

[8388863, 4278190335, 65535]

In [17]:
gps.ColorMap.from_histogram(histogram=hist, color_list=color_ramp)

In [18]:
gps.ColorMap.from_colors(breaks=[0, 250, 1000], color_list=colors_color_ramp)

In [19]:
gps.ColorMap.nlcd_colormap()

In [20]:
break_map = {
    1: 0x00000000,
    2: 0x00000001,
    3: 0x00000002
}

gps.ColorMap.from_break_map(break_map=break_map)

In [21]:
# build using a Histogram
gps.ColorMap.build(breaks=hist, colors=color_ramp)

# It is also possible to pass in the name of Matplotlib color ramp instead of constructing it yourself
gps.ColorMap.build(breaks=hist, colors="viridis")

# build using Colors
gps.ColorMap.build(breaks=colors_color_ramp, colors=colors)

# buld using breaks
gps.ColorMap.build(breaks=break_map)

In [22]:
M.set_center(80.546,6.585, 11)

In [36]:
weighted_histogram = gps.ColorMap.from_histogram(histogram=hist, color_list=color_ramp)
weighted_color_map =gps.ColorMap.nlcd_colormap()

In [37]:
tms = gps.TMS.build(source=pyramided ,
                    display=weighted_color_map)

In [38]:
M.add_layer(TMSRasterData(tms), name="Weighted Layer")